In [ ]:
import checklist
from checklist.editor import Editor
editor = Editor()
import spacy
nlp = spacy.load('en_core_web_sm')

import numpy as np
import random


# from wonderwords import RandomWord, RandomSentence
# word_generator = RandomWord()
# sent_generator = RandomSentence()

import pandas as pd



A

In [ ]:
#Singular noun referent - singular noun mention
#3 stuctures based on syntactic roles: 1) referent=subject & mention=subject 2)referent=object & mention=object 3)referent=object & mention=subject
#Of course other types of structure could be used here, referent=subjecct & mention=object 
#3 variations: 1) mention in a subordinate clause 2) mention in an independent clause 3) cataphora (mention before referent)

#BECAREFUL
#not to create confusion: I don't want the mention to also match another entity in the text.
#In the third structure I don't whant to accidentally create appositions eg. It was delicious, the meal that Anne cooked.
#not to mix the anaphor with another entity. eg. When the anaphor is it, I don't want to have another entity that can take this pronoun
sing = [('After the car crashed, it was completely damaged.', 'The car crashed. It was completely damaged.','After it crashed, the car was completely damaged.'),
 ('The woman bought the watch, but she lost it.', 'The woman bought the watch. However, she lost it.', 'A few minutes after the woman bought it, she lost the watch.'),
 ('The man planted the tree, and a few months later it bloomed.', 'The man planted the tree. A few months later it bloomed.', 'A few months after the man planted it, the tree bloomed.')]

plural = [('The boys left, but they returned.', 'The boys left. However, they returned.', 'They left, but the boys returned.'),
 ('First the mother baked the cupcakes, and then she ate them.', 'First, the mother baked the cupcakes. Then, she ate them.', 'First the mother baked them, and then she ate the cupcakes.'),
 ('The fox attacked the animals, and they ran.', 'The fox attacked the animals. They ran.', 'The fox attacked them, and the animals ran.')]

col = [('The ENTITY ALIAS1, but they ALIAS2.','The ENTITY ALIAS1. However, they ALIAS2.','Even though they ALIAS1, the ENTITY ALIAS2.'),
('The FEMALE_ACTOR ALIAS1 the ENTITY, and she ALIAS2 them.','The FEMALE_ACTOR ALIAS1 the ENTITY. She ALIAS2 them.','The FEMALE_ACTOR ALIAS1 them, and she ALIAS2 the ENTITY.'),
('The BOTH_ACTOR ALIAS1 the ENTITY, and they ALIAS2.','The BOTH_ACTOR ALIAS1 the ENTITY. They ALIAS2.','The BOTH_ACTOR ALIAS1 them, and the ENTITY ALIAS2.')]

#for case 3 of plural and col I don't want to create more confusion by using another plural entity that can also match with they, because then besides number, it will
#also need to look at semantics. This holds for all the parts were there might me a  confusion with the anaphoric expression (pronoun) and the referent.

In [ ]:
#ref_alts = editor.suggest_replace(sent1, entity)[70:120]    #I use both the masking and the wonderwords package
#generator.random_words(20,include_categories=['noun'])
sing_ref = ['car','population','book', 'party', 'ethnicity','preparation','plan',
'stamp','robot','game','pillow','house','fruit','track','elevator','tower','computer','video','mountain','body']

pl_ref = ['slaves','soldiers','guards','people','officers','students','hunters','thieves','villagers','protestors','settlers',
'survivors','refugees','prisoners','wolves','travelers','ghosts','insects','farmers','pirates']

col_ref = ['family','couple','audience','flock','group','team','board','committe','choir','council','jury','staff',
'youth','band','crowd','crew','gang','duo','mob','troupe']

alt_actress = ['woman', 'mother', 'owner', 'author', 'lady', 'bride', 'daughter', 'model', 'sister',
'actress', 'photographer','seller', 'artist', 'teacher','client', 'student','queen', 'princess','witch','hostess']

alt_actor = ['man','farmer','owner','neighbor','soldier','father','student','employee','friend','seller','mayor','hunter','villager','husband','doctor',
'volunteer','resident','reporter','publisher','parent','priest']

alt_both = ['man', 'client','mayor','reporter','resident','president','employee','volunteer','seller','singer','doctor','model','artist','owner','parent','athlete',
'actress','member','soldier','protester']

alt_actors = ['men','soldiers','guards','people']

transitive_verbs_list = ['got', 'opened', 'closed', 'purchased','commited','took','wrote','cooked','ate','changed','summoned','cleaned','collected',
'drank','smoked','chased','helped','kissed','killed','tasted','selected','tried','unlocked','used','broke','needed','turned','raised',
'kept']

In [ ]:
#case1=both_sbj  case2=both_obj  case3=obj_subj
sing1 = {'sentence':[], 'position':[],  'gold':[]}
sing2 = {'sentence':[], 'position':[],  'gold':[]}
sing3 = {'sentence':[], 'position':[],  'gold':[]}

pl1 = {'sentence':[], 'position':[], 'gold':[]}
pl2 = {'sentence':[], 'position':[],  'gold':[]}
pl3 = {'sentence':[], 'position':[],  'gold':[]}

col1 = {'sentence':[], 'position':[],  'gold':[]}
col2 = {'sentence':[], 'position':[],  'gold':[]}
col3 = {'sentence':[], 'position':[],  'gold':[]}

sing_set_df = pd.DataFrame()
pl_set_df = pd.DataFrame()
col_set_df = pd.DataFrame()

final_set_df = pd.DataFrame()

In [ ]:
def replace_verbs(sentence, verb1, verb2):
    masked_sent = sentence.replace(verb1,'{mask}').replace(verb2,'{mask}')
    verb_alternatives = editor.suggest(masked_sent, remove_duplicates = True)[:20]
    #remove the ones that are not suitable
    for tpl in verb_alternatives:
        for word in tpl:
            doc = nlp(word)
            for token in doc:
                if token.pos_ != 'VERB':
                    break
        
        verb_alternatives.remove(tpl)            
        continue
    
    return verb_alternatives[:3]

In [ ]:
#sing - case 1  
sent1 = sing[0][0]
sent2 = sing[0][1]
sent3 = sing[0][2]
verb_a, verb_b = 'crashed','damaged'
entity = 'car'
for alt in sing_ref:
        new_sent1 = sent1.replace(entity, alt)
        verb_alts = replace_verbs(new_sent1, verb_a, verb_b)
        for pair in verb_alts:
                sub = sent1.replace(verb_a, pair[0]).replace(verb_b, pair[1]).replace(entity,alt)
                spacy_gold_sub = 'the '+alt+ ',it'
                ind = sent2.replace(verb_a, pair[0]).replace(verb_b, pair[1]).replace(entity,alt)
                spacy_gold_ind = 'The '+alt+ ',It'
                cat = sent3.replace(verb_a, pair[0]).replace(verb_b, pair[1]).replace(entity,alt)
                spacy_gold_cat = 'it,the '+alt
                sing1['sentence'].extend([sub,ind,cat])
                sing1['position'].extend(['subordinate','independent','cataphora'])
                sing1['gold'].extend([spacy_gold_sub,spacy_gold_ind,spacy_gold_cat])
                

In [ ]:
gold = sing1['gold'][0]
gold = gold.split(",")
gold

In [ ]:
sing1_df = pd.DataFrame.from_dict(sing1)
syntactic_role = ['subj']* len(sing1_df)
sing1_df['syntactic_role']=syntactic_role
len(sing1_df)

In [ ]:
#sing - case 2
sent1 = sing[1][0]
sent2 = sing[1][1]
sent3 = sing[1][2]
verb_a, verb_b = 'bought','lost'
entity = 'watch'
actor = 'woman'
for a in alt_actress:
    new_sent1 = sent1.replace(actor,a)
    #we find replacements for the two verbs in the sentence and the entity
    masked_sent = new_sent1.replace(verb_a,'{mask}').replace(verb_b,'{mask}').replace(entity,'{mask}')
    other_alts = editor.suggest(masked_sent, remove_duplicates=True)[:3]
    
    #we create the sentences
    for group in other_alts:
        sub = sent1.replace(actor,a).replace(verb_a,group[0]).replace(entity,group[1]).replace(verb_b,group[2])
        spacy_gold_sub = 'the '+group[1]+',it'
        ind = sent2.replace(actor,a).replace(verb_a,group[0]).replace(entity,group[1]).replace(verb_b,group[2])
        spacy_gold_ind = 'the '+group[1]+',it'
        cat = sent3.replace(actor,a).replace(verb_a,group[0]).replace(entity,group[1]).replace(verb_b,group[2])
        spacy_gold_cat = 'it,'+ 'the '+group[1]

        sing2['sentence'].extend([sub,ind,cat])
        sing2['position'].extend(['subordinate','independent','cataphora'])
        sing2['gold'].extend([spacy_gold_sub,spacy_gold_ind,spacy_gold_cat])

In [ ]:
sing2_df = pd.DataFrame.from_dict(sing2)
syntactic_role = ['obj']* len(sing2_df)
sing2_df['syntactic_role']=syntactic_role

print(len(sing2_df))

sing_set_df = sing1_df.append(sing2_df)

In [ ]:
#sing - case 3
sent1 = sing[2][0]
sent2 = sing[2][1]
sent3 = sing[2][2]
verb_a, verb_b = 'planted','bloomed'
entity = 'tree'
actor = 'man'
for a in alt_actor:
    new_sent1 = sent1.replace(actor,a)
    #we find replacements for the two verbs in the sentence and the entity
    masked_sent = new_sent1.replace(verb_a, '{mask}').replace(verb_b,'{mask}').replace(entity,'{mask}')
    other_alts = editor.suggest(masked_sent, remove_duplicates=True)[:3]
    for group in other_alts:
        sub = sent1.replace(actor,a).replace(verb_a,group[0]).replace(entity,group[1]).replace(verb_b,group[2])
        spacy_gold_sub = 'the '+group[1]+',it'
        ind = sent2.replace(actor,a).replace(verb_a,group[0]).replace(entity,group[1]).replace(verb_b,group[2])
        spacy_gold_ind = 'the '+group[1]+',it'
        cat = sent3.replace(actor,a).replace(verb_a,group[0]).replace(entity,group[1]).replace(verb_b,group[2])
        spacy_gold_cat = 'it,the '+group[1]

        sing3['sentence'].extend([sub,ind,cat])
        sing3['position'].extend(['subordinate','independent','cataphora'])
        sing3['gold'].extend([spacy_gold_sub,spacy_gold_ind,spacy_gold_cat])
        
        

In [ ]:
sing3_df = pd.DataFrame.from_dict(sing3)
syntactic_role = ['obj_subj']* len(sing3_df)
sing3_df['syntactic_role']=syntactic_role
print(len(sing3_df))
sing_set_df = sing_set_df.append(sing3_df)

specific_cap = ['singular']*len(sing_set_df)
sing_set_df['number']=specific_cap

print(len(sing_set_df)) #3 * 180

In [ ]:
#plural - case 1
sent1 = plural[0][0]
sent2 = plural[0][1]
sent3 = plural[0][2]
verb_a, verb_b = 'left','returned'
entity = 'boys'
for alt in pl_ref:
    new_sent1 = sent1.replace(entity, alt)
    #find alternative verbs
    verb_alts = replace_verbs(new_sent1, verb_a, verb_b)
    for pair in verb_alts:
        sub = sent1.replace(verb_a, pair[0]).replace(verb_b, pair[1]).replace(entity,alt)
        spacy_gold_sub = 'The '+alt+',they'
        ind = sent2.replace(verb_a, pair[0]).replace(verb_b, pair[1]).replace(entity,alt)
        spacy_gold_ind = 'The '+alt+',they'
        cat = sent3.replace(verb_a, pair[0]).replace(verb_b, pair[1]).replace(entity,alt)
        spacy_gold_cat = 'They,'+ 'the '+alt
        pl1['sentence'].extend([sub,ind,cat])
        pl1['position'].extend(['subordinate','independent','cataphora'])
        pl1['gold'].extend([spacy_gold_sub,spacy_gold_ind,spacy_gold_cat])
        

In [ ]:
pl1_df = pd.DataFrame.from_dict(pl1)
syntactic_role = ['subj']* len(pl1_df)
pl1_df['syntactic_role']=syntactic_role
len(pl1_df)

In [ ]:
#plural - case 2
sent1 = plural[1][0]
sent2 = plural[1][1]
sent3 = plural[1][2]
verb_a, verb_b = 'baked','ate'
entity = 'cupcakes'
actor = 'mother'
for a in alt_actress:
    new_sent1 = sent1.replace(actor,a)
    #we find replacements for the two verbs in the sentence and the entity
    masked_sent = new_sent1.replace(verb_a, '{mask}').replace(verb_b,'{mask}').replace(entity,'{mask}')
    other_alts = editor.suggest(masked_sent, remove_duplicates=True)[:3]
    
    #we create the sentences
    for group in other_alts:
        sub = sent1.replace(actor,a).replace(verb_a,group[0]).replace(entity,group[1]).replace(verb_b,group[2])
        spacy_gold_sub = 'the '+group[1]+',them'
        ind = sent2.replace(actor,a).replace(verb_a,group[0]).replace(entity,group[1]).replace(verb_b,group[2])
        spacy_gold_ind = 'the '+group[1]+',them'
        cat = sent3.replace(actor,a).replace(verb_a,group[0]).replace(entity,group[1]).replace(verb_b,group[2])
        spacy_gold_cat = 'them,the '+group[1]

        pl2['sentence'].extend([sub,ind,cat])
        pl2['position'].extend(['subordinate','independent','cataphora'])
        pl2['gold'].extend([spacy_gold_sub,spacy_gold_ind,spacy_gold_cat])
    

In [ ]:
pl2_df = pd.DataFrame.from_dict(pl2)
syntactic_role = ['obj']* len(pl2_df)
pl2_df['syntactic_role']=syntactic_role
print(len(pl2_df))
pl_set_df = pl1_df.append(pl2_df)

In [ ]:
#plural - case 3
sent1 = plural[2][0]
sent2 = plural[2][1]
sent3 = plural[2][2]
verb_a, verb_b = 'attacked','ran'
entity = 'animals'
actor = 'fox'
#we find replacements for the actor
for a in alt_both:
    new_sent1 = sent1.replace(actor,a)
    #we find replacements for the two verbs in the sentence and the entity
    masked_sent = new_sent1.replace(verb_a, '{mask}').replace(verb_b,'{mask}').replace(entity,'{mask}')
    other_alts = editor.suggest(masked_sent, remove_duplicates=True)[:3]
    
    #we create the sentences
    for group in other_alts:
        sub = sent1.replace(actor,a).replace(verb_a,group[0]).replace(entity,group[1]).replace(verb_b,group[2])
        spacy_gold_sub = 'the '+group[1]+',they'
        ind = sent2.replace(actor,a).replace(verb_a,group[0]).replace(entity,group[1]).replace(verb_b,group[2])
        spacy_gold_ind = 'the '+group[1]+',They'
        cat = sent3.replace(actor,a).replace(verb_a,group[0]).replace(entity,group[1]).replace(verb_b,group[2])
        spacy_gold_cat = 'them,the '+group[1]

        pl3['sentence'].extend([sub,ind,cat])
        pl3['position'].extend(['subordinate','independent','cataphora'])
        pl3['gold'].extend([spacy_gold_sub,spacy_gold_ind,spacy_gold_cat])
    


In [ ]:
pl3_df = pd.DataFrame.from_dict(pl3)
syntactic_role = ['obj_subj']* len(pl3_df)
pl3_df['syntactic_role']=syntactic_role
pl_set_df = pl_set_df.append(pl3_df)

specific_cap = ['plural']*len(pl_set_df)
pl_set_df['number']=specific_cap
print(len(pl_set_df))

In [ ]:
#collective - case 1
sent1 = col[0][0]
sent2 = col[0][1]
sent3 = col[0][2]
for ref in col_ref:
    new_sent1 = sent1.replace('ENTITY',ref)
    masked_sent = new_sent1.replace('ALIAS1', '{mask}').replace('ALIAS2','{mask}')
    other_alts = editor.suggest(masked_sent, remove_duplicates=True)[:3]
    
    #we create the sentences
    for group in other_alts:
        zero = group[0]
        if zero.endswith('s'):
            zero = zero.rstrip('s')
        sub = sent1.replace('ENTITY',ref).replace('ALIAS1',zero).replace('ALIAS2',group[1])
        spacy_gold_sub = 'The '+ref+',they'
        ind = sent2.replace('ALIAS1',zero).replace('ALIAS2',group[1]).replace('ENTITY',ref)
        spacy_gold_ind = 'The '+ref+',they'
        cat = sent3.replace('ALIAS1',zero).replace('ALIAS2',group[1]).replace('ENTITY',ref)
        spacy_gold_cat = 'they,the '+ref
        
        col1['sentence'].extend([sub,ind,cat])
        col1['position'].extend(['subordinate','independent','cataphora'])
        col1['gold'].extend([spacy_gold_sub,spacy_gold_ind,spacy_gold_cat])
    

In [ ]:
col1_df = pd.DataFrame.from_dict(col1)
syntactic_role = ['subj']* len(col1_df)
col1_df['syntactic_role']=syntactic_role
len(col1_df)

In [ ]:
#collective - case 2
sent1 = col[1][0]
sent2 = col[1][1]
sent3 = col[1][2]
for ref, actress in zip(col_ref, alt_actress):
    print(actress)
    new_sent1 = sent1.replace('ENTITY',ref).replace('FEMALE_ACTOR',actress)
    masked_sent = new_sent1.replace('ALIAS1', '{mask}').replace('ALIAS2','{mask}')
    other_alts = editor.suggest(masked_sent, remove_duplicates=True)[:20]
    for group in other_alts:
        doc = nlp(group[0])
        for token in doc:
            if token.pos_ != 'VERB':
                break
        other_alts.remove(group)
        continue
    other_alts = other_alts[:3]
    #we create the sentences
    for group in other_alts:
        print(group)
        sub = sent1.replace('FEMALE_ACTOR', actress).replace('ALIAS1',group[0]).replace('ALIAS2',group[1]).replace('ENTITY',ref)
        spacy_gold_sub = 'the '+ref+',them'
        ind = sent2.replace('FEMALE_ACTOR', actress).replace('ALIAS1',group[0]).replace('ALIAS2',group[1]).replace('ENTITY',ref)
        spacy_gold_ind = 'the '+ref+',them'
        cat = sent3.replace('FEMALE_ACTOR', actress).replace('ALIAS1',group[0]).replace('ALIAS2',group[1]).replace('ENTITY',ref)
        spacy_gold_cat = 'them,the '+ref

        col2['sentence'].extend([sub,ind,cat])
        col2['position'].extend(['subordinate','independent','cataphora'])
        col2['gold'].extend([spacy_gold_sub,spacy_gold_ind,spacy_gold_cat])

In [ ]:
col2_df = pd.DataFrame.from_dict(col2)
syntactic_role = ['obj']* len(col2_df)
col2_df['syntactic_role']=syntactic_role
print(len(col2_df))

col_set_df = col1_df.append(col2_df)

In [ ]:
#collective - case 3
sent1 = col[2][0]
sent2 = col[2][1]
sent3 = col[2][2]
for ref, actor in zip(col_ref, alt_both):
    new_sent1 = sent1.replace('ENTITY',ref).replace('BOTH_ACTOR',actor)
    masked_sent = new_sent1.replace('ALIAS1', '{mask}').replace('ALIAS2','{mask}')
    other_alts = editor.suggest(masked_sent, remove_duplicates=True)[:3]
    
    #we create the sentences
    print(actor)
    for group in other_alts:
        print(group)
        sub = sent1.replace('BOTH_ACTOR', actor).replace('ALIAS1',group[0]).replace('ALIAS2',group[1]).replace('ENTITY',ref)
        spacy_gold_sub = 'the '+ref+',they'
        ind = sent2.replace('BOTH_ACTOR', actor).replace('ALIAS1',group[0]).replace('ALIAS2',group[1]).replace('ENTITY',ref)
        spacy_gold_ind = 'the '+ref+',They'
        cat = sent3.replace('BOTH_ACTOR', actor).replace('ALIAS1',group[0]).replace('ALIAS2',group[1]).replace('ENTITY',ref)
        spacy_gold_cat = 'them,the '+ref

        col3['sentence'].extend([sub,ind,cat])
        col3['position'].extend(['subordinate','independent','cataphora'])
        col3['gold'].extend([spacy_gold_sub,spacy_gold_ind,spacy_gold_cat])

In [ ]:
col3_df = pd.DataFrame.from_dict(col3)
syntactic_role = ['obj_subj']* len(col3_df)
col3_df['syntactic_role']=syntactic_role
print(len(col3_df))
col_set_df = col_set_df.append(col3_df)

specific_cap = ['collective']*len(col_set_df)
col_set_df['number']=specific_cap

print(len(col_set_df)) #3 * 180

In [ ]:
final_set_df = sing_set_df.append(pl_set_df)
final_set_df = final_set_df.append(col_set_df)

path = '/Users/vicky/Desktop/Master/Period4/NLP/resit/data/number_position_synt_role.csv'

final_set_df.to_csv(path)

In [ ]:
#STATISTICS
num_df = pd.read_csv(path)

In [ ]:
print(len(num_df)) #number of total instances

In [ ]:
print(540/3) #for each number category each grammatical role has 180 instances.
# a grammatical role has 540 instances across all number in total